# Intro

This is the first script in the data preprocessing pathway. The final product is a tidy dataset annotated with the metadata defined in the metadata.json file.
All files associated with each round of AL are prefixed with the individual ID for that round of learning. e.g. ALTE001. 

# Order of Operations

## 1. The splitting of the chromatics.

* It imports a "_raw.csv" .csv raw data file from the BMG Omega platereader & the metadata .json.
* Determines how many chromatics have been measured, how many minutes it was recording for and moves the columns down into rows to separate out the data for each channel and gain.
* Saves the resulting format as .csv suffixed with "_parsed_data.csv"


In [1]:
import pandas as pd
import numpy as np
import math
import os, sys, shutil
import json

# Defining some functions

In [2]:


def print_all_df(df):
    """
    This function simply prints an entire pandas dataframe by changing the global pandas settings,
    printing and then reverting the settings.
    """
    # Permanently changes the pandas settings
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', -1)
    
    # All dataframes hereafter reflect these changes.
    display(df)
    
    print('**RESET_OPTIONS**')
    
    # Resets the options
    pd.reset_option('all')
    



# Define the AL Round Prefix

This is the AL round ID that the script will use to determine with files to import and what to name the products.

e.g. "ALTE001"

In [3]:
prefix = "ALTE006"


# Define the end time

Enter the exact string of the last time point measured in the rawdata file.

In [4]:
end_time_string = "11 h 58 min"

# Import and exact the metadata

In [5]:
# navigate into the /app directory in the linux distribution of the docker container
os.chdir('/app')

# use the prefix to get the name of the metadata file
metadata_file_string = prefix + "_experiment_metadata.json"
metadata_file_string

'ALTE006_experiment_metadata.json'

In [7]:

# move into the metadata folder
os.chdir('/app/data/experiment_metadata_files')
print(os.getcwd())

# Opening JSON file 
f = open(metadata_file_string) 

# returns JSON object as a dictionary 
experiment_metadata = json.load(f) 

# unpack dictionary into sub dictionaries and variables
well_designation = experiment_metadata['well_designation']

# save various directory elements as named variables
metadataheader = experiment_metadata['metadataheader']
chromatics_designations = experiment_metadata['chromatics_designations']
gap = experiment_metadata['gap']
metadata = experiment_metadata['metadata']

/app/data/experiment_metadata_files


# Import and extract the raw data

This cell basically imports the raw data, splits it up a bit and determines:

* The number of chromatics
* The last time point measured

In [8]:
# use the prefix to get the raw data file name
rawdata_file_string = prefix + "_raw.csv"

# Navigate into the raw data directory
os.chdir('/app/data/raw_data_files/')

# import the raw data
df = pd.read_csv(rawdata_file_string, header=None)

In [9]:
# Check to see if raw datafile is in the old or new format

#if not manually rearranged...
if df.iloc[10,0] == "Well":
    
    # save the actual data
    just_the_data = df.iloc[10:,:].copy()
    
    # grab the metadata column
    just_the_column = df.iloc[:,0].copy()
    
    
    # do all the manual assigning
    df.iloc[0,1] = just_the_column.loc[1]
    df.iloc[0,2] = just_the_column.loc[2]
    df.iloc[1,0] = just_the_column.loc[3]
    df.iloc[1,1] = just_the_column.loc[4]
    df.iloc[1,2] = just_the_column.loc[5]
    df.iloc[2,0] = just_the_column.loc[6]
    df.iloc[3,0] = just_the_column.loc[9]
    df.iloc[4,0] = just_the_column.loc[7]
    df.iloc[5,0] = just_the_column.loc[7]

    
    
    # get the rearranged metadata top
    just_the_top = df.iloc[:5,:]
    
    
    #build the new df
    df = pd.concat([just_the_top, just_the_data], axis=0)
    df = df.reset_index(drop=True)
    
    
    print_all_df(df)

    # if it's already been arranged, pass
else:
    pass
    

/tmp/ipykernel_575/3947327734.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721
0,User: USER,Path: C:\Program Files (x86)\BMG\Omega\User\Data\,Test ID: 584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

**RESET_OPTIONS**


/usr/local/lib/python3.10/dist-packages/pandas/_config/config.py:653: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/pandas/_config/config.py:653: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [10]:
print(df)

# segregate the metadata
df_head = df.iloc[:4,:]


df_head = df_head.set_axis(np.arange(0,df_head.shape[1],1), axis=1)

# these are the actual data
df_samples = df.iloc[5:,:2]

# gets the row that the time is in
timelist = df.iloc[6,:]
timelist = timelist.reset_index(drop=True)

last_timepoint = timelist.iloc[-1]

# gets the number of chromatics by counting how many times the times are repeated
number_of_chromatics = timelist.value_counts()[0]

                                  0    \
0                          User: USER   
1                Test Name: AP_MS_GFP   
2   Fluorescence (FI), multichromatic   
3    Used kinetic range(s): Range 1;    
4                                 NaN   
5                                Well   
6                                 NaN   
7                                 B03   
8                                 B05   
9                                 B07   
10                                B09   
11                                B11   
12                                B13   
13                                B15   
14                                B17   
15                                B19   
16                                B21   
17                                D03   
18                                D05   
19                                D07   
20                                D09   
21                                D11   
22                                D13   
23              

# Converts the time string into a integer minute list.
The plate reader spits out the timepoints in strings that come in two different formats based on god knows what reason. Regardless, if it's the '0 h 0 min' format then there's a long winded and complicated parser. If it's the ' 0 mins' format, it's much more straight forward.

In [11]:

# check to see which time format the data is in.

# if it's '0 h 0 min'....
if (last_timepoint.__contains__('h')):
    
    
    print("I think the time format is '0 h 0 min'")
    
    
    print(timelist[timelist == '0 h '].index)
    print('length of index thing')
    print(len(timelist[timelist == '0 h '].index))
    print('')

    chromatics_list = [df_head]


    for i, v in enumerate(timelist[timelist == '0 h '].index):
        start = v


        if (i+1) >= len(timelist[timelist == '0 h '].index):

            chromatic = df.iloc[4:, start:]
            chromatic.insert(loc=0, column="Content", value=df.iloc[4:,1])
            chromatic.insert(loc=0, column="Well", value=df.iloc[4:,0])
            chromatic = chromatic.set_axis(np.arange(0,chromatic.shape[1],1), axis=1)
            print(chromatic)
            chromatics_list.append(chromatic)

        else:
            end = timelist[timelist == '0 h '].index[i+1]

            chromatic = df.iloc[4:, start:end]
            chromatic.insert(loc=0, column="Content", value=df.iloc[4:,1])
            chromatic.insert(loc=0, column="Well", value=df.iloc[4:,0])
            #add spacer rows
            chromatic = chromatic.append(pd.Series(), ignore_index=True)
            chromatic = chromatic.append(pd.Series(), ignore_index=True)


            chromatic = chromatic.set_axis(np.arange(0,chromatic.shape[1],1), axis=1)
            chromatics_list.append(chromatic)

    final = pd.concat(chromatics_list)

    final.reset_index(inplace=True, drop=True)

    mask = final.iloc[6,:] == end_time_string
    mask = mask.values

    twelve_hr_index = final.iloc[6,mask].index

    twelve_hr_index = twelve_hr_index[0]

    twelve_hr_index = twelve_hr_index + 1

    final = final.iloc[:,:twelve_hr_index]

    # making the new name by prefix +  _parsed_dataset
    new_name = prefix+'_parsed_dataset.csv'


    #######################################################################

    print(os.getcwd())
    path = "/app/data/parsed_data_files/"
    # make directory for sticking the output in
    if os.path.isdir(path) == False:
        os.mkdir(path, mode=0o777)

    os.chdir(path)
    final.to_csv(new_name, header=False, index=False)


    #navigate home for neatness
    os.chdir('/app')

    
    
else:
    
    #print("I think the time format is '0 mins'")
    
    #make a python list from the series
    timelist = list(timelist)
    
    # drop nan and time
    timelist = timelist[2:]
    
    #remove the last four characters of every string
    timelist = [sub[:-5] for sub in timelist]
    
    #make them all numerical
    timelist = [int(sub) for sub in timelist]
    
    # get max time point
    max_time = max(timelist)
    
    # get index of max values
    res = [x for x, z in enumerate(timelist) if z == max_time] 
    
    # add 2 back to account for nan and time removed earlier  plus one for the indexing factor
    res = [x+3 for x in res]
    
    # we now have the indexes and are ready to slice
    
    # first off grab the well and sample names
    well_and_sample_names = df.iloc[5:,:2].copy()
    well_and_sample_names = well_and_sample_names.reset_index(drop=True)

    
    # iterate over the indexes
    for i, indexer in enumerate(res):
        
        # if it's the first one
        if i == 0:
            
            #intialise new df from the metadata before
            #add spacer rows
            reshuffled = df_head.copy()
            reshuffled = reshuffled.append(pd.Series(), ignore_index=True)
            
            # grab the first one
            chromatic = df.iloc[5:,2:indexer].copy()
            chromatic = chromatic.reset_index(drop=True)
            
            # add the well and sample names
            chromatic = pd.concat([well_and_sample_names, chromatic], axis=1)
            chromatic = chromatic.reset_index(drop=True)
            
            # append to reshuffled
            reshuffled = reshuffled.append(chromatic, ignore_index=True)
            reshuffled = reshuffled.reset_index(drop=True)
            
            # save v for slicing the next bit
            last_indexer = indexer
            
            
        # if more than the first    
        else:
            
            # add the gap spacers
            for i in range(0,gap,1):
                print(i)
                reshuffled = reshuffled.append(pd.Series(), ignore_index=True)

            
            # grab the next one
            chromatic = df.iloc[5:, last_indexer: indexer].copy()
            chromatic = chromatic.reset_index(drop=True)
            
            
            # add the well and sample names
            chromatic = pd.concat([well_and_sample_names, chromatic], axis=1)
            chromatic = chromatic.reset_index(drop=True)
            
            # resets the column index
            chromatic = chromatic.T.reset_index().T.reset_index(drop=True)
            chromatic = chromatic.iloc[1:,:]
            
            #print(chromatic)
            
            # append to reshuffled
            reshuffled = reshuffled.append(chromatic, ignore_index=True)
            reshuffled = reshuffled.reset_index(drop=True)
            
            # save v for slicing the next bit
            last_indexer = indexer
        
    
    
    # carve off any overhang Nans
    
    reshuffled = reshuffled.iloc[:,:res[0]-1]
    
    
    # making the new name by replacing raw with parsed_dataset
    new_name = prefix + "_parsed_dataset.csv"
    
    #######################################################################

    print(os.getcwd())
    path = "/app/data/parsed_data_files/"
    # make directory for sticking the output in
    if os.path.isdir(path) == False:
        os.mkdir(path, mode=0o777)

    os.chdir(path)
    reshuffled.to_csv(new_name, header=False, index=False)


    #navigate home for neatness
    os.chdir('/app')

            
        

I think the time format is '0 h 0 min'
Int64Index([2, 362], dtype='int64')
length of index thing
2

     0           1                     2                     3    \
4    NaN         NaN                   NaN                   NaN   
5   Well     Content  Raw Data (485/520 1)  Raw Data (485/520 1)   
6    NaN        Time                  0 h              0 h 2 min   
7    B03   Sample X1                    80                    79   
8    B05   Sample X2                    79                    80   
9    B07   Sample X3                    82                    81   
10   B09   Sample X4                    79                    76   
11   B11   Sample X5                    83                    76   
12   B13   Sample X6                    81                    81   
13   B15   Sample X7                    80                    78   
14   B17   Sample X8                    78                    79   
15   B19   Sample X9                    81                    76   
16   B21  Sample

/tmp/ipykernel_575/3722162469.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  chromatic = chromatic.append(pd.Series(), ignore_index=True)
/tmp/ipykernel_575/3722162469.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  chromatic = chromatic.append(pd.Series(), ignore_index=True)
/tmp/ipykernel_575/3722162469.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  chromatic = chromatic.append(pd.Series(), ignore_index=True)
/tmp/ipykernel_575/3722162469.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  chromatic = chromatic.append(pd.Series(), ignore_index=True)


# Tidy Data Script

This section actually does the metadata annotation

In [12]:

def import_data(filename):
    """ 
    Imports the raw .csv
    Segregates the metadata out and stores it along with the actual data in a dictionary
    """
    
    data = pd.read_csv(filename, header=None)
   

    date = data.iloc[0,1]
    metadata = data.iloc[:3,:3]

    data = data.iloc[5:,:]
    data.reset_index(inplace=True)
    data = data.iloc[:,1:]
    

    keys = ['date', 'metadata','raw_data']
    values = [date, metadata, data]

    data_dict = {keys: values for keys, values in zip(keys, values)}
    return data_dict

In [13]:


def slice_dataframe_based_on_experimental_number(num_of_experiments, num_of_standards, raw_data):
    """"Slices data frame based on the experimental number."""
    """ Takes in the experimental and standards numbers as determined in the determine_experimental_data(), the gap number from metadata.json """

    """ Includes negative control in experimental number """

    """ Returns a python list of dataframes for each chromatic."""


    # initial empty list
    chromatic_list = []
    
    # first chromatic
    # The chromatic slice factor is effectively the number of rows in each chromatic.
    chromatic_slicefactor = num_of_experiments + metadataheader + num_of_standards  

    # just take the first chromatic at the top of the raw data DF
    chromatic_1_slice = raw_data.iloc[:chromatic_slicefactor,:]
    #append to list 
    chromatic_list.append(chromatic_1_slice)
    
    #iterate over the rest
    for i in range(1, number_of_chromatics,1):
        
        # move down!
        old_factor = chromatic_slicefactor + gap
        
        chromatic_slicefactor = chromatic_slicefactor + gap + num_of_experiments + metadataheader + num_of_standards
        chromatic_slice = raw_data.iloc[old_factor : chromatic_slicefactor,:]
        chromatic_list.append(chromatic_slice)
    
    return chromatic_list 



In [14]:

def determine_experimental_number():
    """
    Works out how many wells are used for actual experiments.
    Works by iterating through the well metadata dictionary and simply counting the standards and experiments.
    Returns the numbers for both.
    """
    
    num_of_experiments = 0
    num_of_standards = 0
    #go through the well_designation dict and if a reaction type is not a standards, plus 1 to num of experiments
    for well in well_designation.keys():
        if well_designation[well]['Reaction Type'] != 'Standard':
            num_of_experiments +=1
        else:
            num_of_standards +=1

    return num_of_experiments, num_of_standards


In [15]:
# navigate into processed_data_files and grab the parsed dataset
os.chdir('/app/data/parsed_data_files')

In [16]:
##########################################################################################

chromatic_name_list_for_saving = []

## get the dataset name
dataset_name = prefix + "_parsed_dataset.csv"

# import the data using the function defined at the top
raw_package_dict = import_data(dataset_name)

#get raw data from dictionary
raw_data = raw_package_dict['raw_data']

# extracting experimental details using the dict
num_of_experiments, num_of_standards = determine_experimental_number()
print(num_of_experiments)

# make the list
chromatic_list = slice_dataframe_based_on_experimental_number(num_of_experiments,num_of_standards, raw_data)



20


# Organise time and refactor into integer minutes

In [17]:
# get timelist to determine time format
minute_list = chromatic_list[0].iloc[1,:]

# get 0h 0min
first_timepoint = minute_list.loc[2]



# check to see which time format the data is in.
# if does contain h.....
if "h" in first_timepoint:

    #get time list
    time = list(raw_data.iloc[raw_data[raw_data.iloc[:,1]=='Time'].index[0],:])
    #delete inital nan

    if math.isnan(float(time[0])) == True:
        del time[0]
        del time[0]
        print('Time list is ready, deleted nan and time')
    elif time[0] == 'Time':
        del time[0]
        print('Time list is ready, deleted time')
    elif time[0] == '0 h ':
        print('Time list is ready')
    else:
        print('Time list is weird')


    #remove nans
    time = [x for x in time if str(x) != 'nan']
    
    # generate the minute list

    minute_list= []

    for i in time:
        #sets the first character of the string as a numeric
        a = int(str(i)[0])

        if len(i) == 4:
            # if the time is a single hour, e.g. '2 h', multiply the first character by 60 and return
            hour = int(str(i)[0]) * 60
            # addition
            total_time = hour
            # add to list
            minute_list.append(total_time)

        if len(i) == 5:
            # if the time is a single hour, e.g. '2 h', multiply the first character by 60 and return
            hour = int(str(i)[0:2]) * 60
            # addition
            total_time = hour
            # add to list
            minute_list.append(total_time)

        elif len(i) == 9:
            # multiply the first character by 60
            hour = int(str(i)[0]) * 60
            # save the fifth character
            minute = int(str(i)[4])
            # addition
            total_time = hour + minute
            # add to list
            minute_list.append(total_time)

        elif len(i) == 10:
            #if the first number is a less than 10 (1 not 11)
            if i[1].isspace():
                # multiply the first character by 60
                hour = int(str(i)[0]) * 60
                # save the fifth and sixth characters
                minute = int(str(i)[4:6])
                # addition
                total_time = hour + minute
                # add to list
                minute_list.append(total_time)
            else:
                # multiply the first character by 60
                hour = int(str(i)[0:2]) * 60
                # save the fifth and sixth characters
                minute = int(str(i)[5])
                # addition
                total_time = hour + minute
                # add to list
                minute_list.append(total_time)

        elif len(i) == 11:
            # multiply the first character by 60
            hour = int(str(i)[0:2]) * 60
            # save the fifth and sixth characters
            minute = int(str(i)[5:7])
            # addition
            total_time = hour + minute
            # add to list
            minute_list.append(total_time)
            
else:
    
## else if it doesn't contain h....

    # convert to python list
    minute_list = list(minute_list)
    
    #delete first two, nan and time
    minute_list = minute_list[2:]
    
    # delete any nans
    minute_list =  [x for x in minute_list if  pd.isnull(x) == False]
    
    #remove the last four characters of every string
    minute_list = [sub[:-5] for sub in minute_list]
    
    
    #make them all numerical
    minute_list = [int(sub) for sub in minute_list]
    
    
    
############################################################################################## 


# add time and thing back
minute_list.insert(0,'Time (Mins)')
minute_list.insert(0,'Sample')

# great
print(minute_list)


Time list is ready, deleted nan and time
['Sample', 'Time (Mins)', 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 300, 302, 304, 306, 308, 310, 312, 314, 316, 318, 320, 322, 324, 326, 328, 330, 332, 334, 336, 338, 340, 342, 344, 346, 348, 350, 352, 354, 356, 358, 360, 362, 364, 366, 368, 370, 372, 374, 376, 378, 380, 382, 384, 386, 388, 390, 392, 394

In [18]:

#store the chromatic names for saving the data
for i,v in enumerate(chromatic_list):

    
    #get the whole string contatining the chromatic name
    chromatic_name = chromatic_list[i].iloc[0,2]
    
    print(chromatic_name)
    
    # get index of the '(' character in the string'
    chromatic_name_index = chromatic_list[i].iloc[0,2].index('(')
    
    # trim the string to include all the characters after the ( 
    chromatic_name = chromatic_name[chromatic_name_index+1:]
    
    # get rid of the last character )
    chromatic_name = chromatic_name[:-1]
    
    chromatic_name = chromatic_name.replace(" ", "_")
    chromatic_name = chromatic_name.replace("/", "_")
    
    chromatic_name_list_for_saving.append(chromatic_name)

Raw Data (485/520 2)
Raw Data (485/520 1)


# Get the name of the chromatic to annotate with later

# Put the integer time list back on each chromatic df

In [19]:
def add_time(df, time_list):

    #check index correct
    df.reset_index(inplace=True)
    df = df.iloc[:,1:]
    
    #delete spectral and time row if necessary
    while df.iloc[0,1] != "Time":
        df = df.iloc[1:,:]
        df.reset_index(inplace=True)
        df = df.iloc[:,1:]

    #insert the minute lists
    df.iloc[0,:] = minute_list

    return df

for i,v in enumerate(chromatic_list):
    chromatic_list[i] = add_time(chromatic_list[i], minute_list)
    

In [20]:
def reorder(df):
    
    #get the row that is to be the columns
    column_row = df.iloc[df.loc[df.iloc[:,1]=='Time (Mins)',:].index[0]]
    
    #rename columns
    df.columns = column_row
    
    
    df = df.iloc[1:,:]
    
    # save sample and time for the melt
    keep_these = column_row[:2]
    #drop Sample and Time
    cols_time = column_row[2:]

    df = pd.melt(df,
            
            id_vars=keep_these,
            
            value_vars=cols_time,
            
            var_name='Time (Minutes)',
            value_name='RFUs'
           )
    
    # rename Sample to well and Time (Mins to Sample Id)
    df.rename(columns = {"Sample": "Well", "Time (Mins)":"Sample ID"}, inplace = True)
    
    ##replace in categories list
    #categories[categories.index('Time (Mins)')] = 'Time (Minutes)'
    #categories[categories.index('Sample')] = 'Sample ID'
    #categories = categories + ['Well']

    
    return df


# iterate over the chromatic list and...
for i,v in enumerate(chromatic_list):
        
        # call reorder() on each chromatic
        chromatic_list[i] = reorder(chromatic_list[i])
        
        #Add chromatics and gains
        print(i)
        print(chromatic_name_list_for_saving[i])
        print(chromatics_designations)
        
        
        # look up the relevent gain setting (1500 or whatever) in the chromatic designations metadata dictionary using the
        # appropriate key (e.g. 485_520_1)
        chromatic_list[i]['Gain Setting'] = chromatics_designations[chromatic_name_list_for_saving[i]]
        
        # save the key
        chromatic_list[i]['Chromatic Settings'] = chromatic_name_list_for_saving[i]
        
        # chop off the "_1" bit 
        chromatic_list[i]['Chromatic Settings'] = chromatic_list[i]['Chromatic Settings'].str.replace('_'+str(i+1), '')
        
        ##### in this bit we're going to use the chromatic setting (485_520) to determine what the expression product was and 
        # then annotating appropriate
        
        if chromatic_name_list_for_saving[i] == "485_520":
            chromatic_list[i]['Expression Product'] = "GFP"
            
        elif chromatic_name_list_for_saving[i][:-2] == "485_520":
            chromatic_list[i]['Expression Product'] = "GFP"
        
        elif chromatic_name_list_for_saving[i][:-2] == "635_680":
            
            chromatic_list[i]['Expression Product'] = "Malachite Green"
            
        else:
            
            chromatic_list[i]['Expression Product'] = "Unknown"
               

0
485_520_2
{'485_520_1': '800', '485_520_2': '1500'}
1
485_520_1
{'485_520_1': '800', '485_520_2': '1500'}


# Time to compile the chromatic list DFs into a tidy data set

In [21]:
# Concatenate
tidy_data = pd.concat(chromatic_list, axis=0, sort=False)

# remove duplicated rows and or columns
tidy_data = tidy_data.loc[:,~tidy_data.columns.duplicated()]


# Adding the Well-Specific Metadata

Uses the well specific metadata to annotate each well.

In [22]:
def get_metadata_categories_from_well_designation(dictionary=well_designation):
    
    meta_data_arr = np.array([])

    for well in dictionary.keys():

        meta_data_arr = np.append(meta_data_arr, np.array(list(dictionary[well].keys())))

    meta_data_arr = np.unique(meta_data_arr)

    return meta_data_arr

meta_data_arr = get_metadata_categories_from_well_designation()


In [23]:
def assign_metadata_by_well(df, metadatacolumn):
    
    """takes the metadata given, 
    creates a column then iterates over the wells,
    looks up that well in the well_designation dictionary,
    if that well has the metadata given then it adds it to that well's entry in the new column,
    if not then it leaves it blank.
    returns the dataframe"""
    
    #creates blank column
    df[metadatacolumn] =''

    #iterates over the wells
    for well in df['Well'].unique():
        
        if metadatacolumn in well_designation[well]:
            
            df.loc[df['Well'] == well, metadatacolumn] = well_designation[well][metadatacolumn]
            
        else:
            
            df.loc[df['Well'] == well, metadatacolumn] = ''
    
    return df

for meta in meta_data_arr:
    tidy_data = assign_metadata_by_well(tidy_data, meta)
tidy_data

,Well,Sample ID,Time (Minutes),RFUs,Gain Setting,Chromatic Settings,Expression Product,Amplicon DNA Template,Energy Solution,Experiment #,...,lysate_aspirate_height_inc,lysate_aspirate_height_init,substrates_aspirate_height_inc,substrates_aspirate_height_init,wax_air_gap,wax_dispense_height,wax_dispense_volume,wax_disposal_volume,wax_new_tip,wax_touch_tip
0,B03,Sample X1,0,3148,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False
1,B05,Sample X2,0,3045,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False
2,B07,Sample X3,0,3091,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False
3,B09,Sample X4,0,3056,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False
4,B11,Sample X5,0,3189,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,D13,Sample X16,718,207,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False
7196,D15,Sample X17,718,207,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False
7197,D17,Sample X18,718,167,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False
7198,D19,Sample X19,718,108,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,0.4,4.5,0.6,8.6,20,-3,35,30,never,False


# Experiment-Wide MetaData

Since this is experiment-wide, we can just make a new column and set it for all.

In [24]:
for meta in metadata.keys():
    tidy_data[meta] = metadata[meta]
tidy_data

,Well,Sample ID,Time (Minutes),RFUs,Gain Setting,Chromatic Settings,Expression Product,Amplicon DNA Template,Energy Solution,Experiment #,...,wax_air_gap,wax_dispense_height,wax_dispense_volume,wax_disposal_volume,wax_new_tip,wax_touch_tip,Reaction Temperature (°C),Performed by,Instrument,Plate Reader Run #
0,B03,Sample X1,0,3148,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6
1,B05,Sample X2,0,3045,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6
2,B07,Sample X3,0,3091,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6
3,B09,Sample X4,0,3056,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6
4,B11,Sample X5,0,3189,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,D13,Sample X16,718,207,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6
7196,D15,Sample X17,718,207,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6
7197,D17,Sample X18,718,167,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6
7198,D19,Sample X19,718,108,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,20,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6


In [25]:
#print(tidy_data)
tidy_data['Assay Date'] = raw_package_dict['metadata'].iloc[1,1][6:]
tidy_data['Assay Date'] = pd.to_datetime(tidy_data['Assay Date'])
tidy_data

/usr/local/lib/python3.10/dist-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '21/06/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)


,Well,Sample ID,Time (Minutes),RFUs,Gain Setting,Chromatic Settings,Expression Product,Amplicon DNA Template,Energy Solution,Experiment #,...,wax_dispense_height,wax_dispense_volume,wax_disposal_volume,wax_new_tip,wax_touch_tip,Reaction Temperature (°C),Performed by,Instrument,Plate Reader Run #,Assay Date
0,B03,Sample X1,0,3148,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21
1,B05,Sample X2,0,3045,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21
2,B07,Sample X3,0,3091,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21
3,B09,Sample X4,0,3056,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21
4,B11,Sample X5,0,3189,1500,485_520_2,GFP,s70_deGFP_MGA,AP_ES_004,7,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,D13,Sample X16,718,207,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21
7196,D15,Sample X17,718,207,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21
7197,D17,Sample X18,718,167,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21
7198,D19,Sample X19,718,108,800,485_520_1,GFP,s70_deGFP_MGA,Michael_ES_001,8,...,-3,35,30,never,False,37,AP & MJS,BMG POLARstar Omega,6,2022-06-21


# Save to CSV

In [26]:

path = "/app/data/tidy_data_files/"

# make directory for sticking the output in
if os.path.isdir(path) == False:
    os.mkdir(path, mode=0o777)
    
    
#navigate to tidy_data_files
os.chdir(path)

tidy_data.to_csv(prefix+"_tidy_data.csv", header=True, index=False)


#navigate home for neatness
os.chdir('/app')